In [140]:
# Importing Libraries Required to import API's and Graph/Interface with Data:
import os
import numpy as np
import pandas as pd
import json as js
import requests
import http.client
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
from pathlib import Path
from SplitTool import splitDataFrameList

%matplotlib inline

In [141]:
# Loading .env environment variable:
load_dotenv()

True

In [142]:
# Setting API key for Attom Property Snapshot API:
attom_api_key = os.getenv('ATTOM_API_KEY')

# Verifying that key was passed through accurately:
print(f"Attom Key type: {type(attom_api_key)}")

Attom Key type: <class 'str'>


In [143]:
# Defining csv path to read us cities data csv which includes zip code data:
uscities_csv = Path("../Data/uscities.csv")

# imorting uscities.csv and extracting zip code data to utilize in API, data selections:
read_df = pd.read_csv(uscities_csv)

# isolating the city and zip columns from read_df and assigning to new df:
zip_df = read_df[{'zips', 'city'}]

# Breaking out multiple zips in single cell to individual row while keeping city element:
zip_df_new = (zip_df.set_index('city')
   .apply(lambda x: x.str.split(' ').explode())
   .reset_index())

zip_df_new

,city,zips
0,New York,11229
1,New York,11226
2,New York,11225
3,New York,11224
4,New York,11222
...,...,...
46276,Caputa,57703
46277,Hamill,57534
46278,Loomis,57301
46279,Bijou Hills,57369


In [155]:
# Connecting ATTOM API to my python code:
conn = http.client.HTTPSConnection("api.gateway.attomdata.com")

# Accepting headers as per the ATTOM API documentation:
headers = { 
    'accept': "application/json", 
    'apikey': attom_api_key, 
}

In [157]:
# Initiating API request to pull property data from Attom API:

# Requst for proprety detail as it pertains to New York:
conn.request("GET", "/propertyapi/v1.0.0/property/detail?postalcode=10004&page=1&pagesize=100", headers=headers)

# Request for listing of properties within a specific zip code:
#conn.request("GET", f"/propertyapi/v1.0.0/property/address?postalcode={api_zip}&page=1&pagesize=100", headers=headers)

# assigning response to variable "res" and reading the data and assigning to variable "data":
res = conn.getresponse() 
data = res.read()